In [22]:
import pandas as pd
import torch
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import karman
from karman.nn import *
import os
import argparse
from karman import Benchmark

In [23]:
def load_model(model_path,
               data_directory='/home/jupyter/karman-project/data_directory'):
  
    print('Loading Data')
    model_opt=torch.load(model_path)['opt']
    print(model_opt)

    dataset=karman.ThermosphericDensityDataset(
        directory=data_directory,
        features_to_exclude_thermo=model_opt.features_to_exclude_thermo.split(','),
        create_cyclical_features=True,
        max_altitude=model_opt.max_altitude,
    )

    print('Loading Model')

    if model_opt.model == 'NoFism2FlareAndDailyAndOmniFeedForward':
        model = SimpleNN(dropout=model_opt.dropout).to(dtype=torch.float32)

    state_dict = torch.load(os.path.join(model_path))['state_dict']
    #Sanitize state_dict key names
    for key in list(state_dict.keys()):
        if key.startswith('module'):
        # Model was saved as dataparallel model
            # Remove 'module.' from start of key
            state_dict[key[7:]] = state_dict.pop(key)
        else:
            continue
    model.load_state_dict(state_dict)

    # infer model fold from path given
    # assumes model follows format: 'model_name_fold_x_seed_0' and uses 'x' as the fold
    fold = int(model_path.split('fold')[-1][1])

    test_month_idx = 2 * (int(fold) - 1)
    validation_month_idx = test_month_idx + 2
    dataset._set_indices(test_month_idx=[test_month_idx], validation_month_idx=[validation_month_idx])

    #Need to do this if using the LazyLinear Module (avoids having to hard code input layers to a linear layer)
    dummy_loader = torch.utils.data.DataLoader(dataset.validation_dataset(),
                                               batch_size=2,
                                               pin_memory=False,
                                               num_workers=0,
                                               drop_last=True)
    model.forward(next(iter(dummy_loader)))
    return model, dataset

In [25]:
def benchmark_model(model_path,
                    batch_size=512,
                    num_workers=30,
                    data_directory='/home/jupyter/karman-project/data_directory',
                    output_save='/home/jupyter/karman-project/benchmark_results',
                    model_name='model'):
    """
    Given a model path, this method loads the model and benchmarks for that stored model.
    """
    model, dataset = load_model(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    return Benchmark(
        batch_size=batch_size,
        num_workers=num_workers,
        data_directory=data_directory,
        output_directory=output_save,
        model_name=model_name).evaluate_model(dataset, model)

In [31]:
def benchmark_model_folder(folder_path,
                           batch_size=512,
                           num_workers=30,
                           data_directory='/home/jupyter/karman-project/data_directory',
                           output_save='/home/jupyter/karman-project/benchmark_results',
                           model_name='model'):
    """
    This method takes all models in a folder and benchmarks them. These models should all be
    the same model but trained on different folds.

    It assumes the models end in 'fold_x_seed_y'
    """
    model_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f[-6:-2] == 'seed']
    print('Models to load:', model_paths)
    results = []

    for model_path in model_paths:
        results.append(
            benchmark_model(model_path,
                            batch_size=batch_size,
                            num_workers=num_workers,
                            data_directory=data_directory,
                            output_save=output_save,
                            model_name=model_name)
            )

    benchmark_results = pd.concat(results, ignore_index=True)
    benchmark_results_mean = benchmark_results.dropna().groupby(['Model', 'Metric Type', 'Condition']).mean()
    benchmark_results_std = benchmark_results.dropna().groupby(['Model', 'Metric Type', 'Condition']).std()
    return benchmark_results_mean, benchmark_results_std




In [32]:
benchmark_results_means, benchmark_results_stds = benchmark_model_folder('/home/jupyter/karman-project/kml_results/')
for row in benchmark_results_means.iterrows():
    print(row[0][0], row[0][1], row[0][2], row[1]['Metric Value'])

Models to load: ['/home/jupyter/karman-project/kml_results/kml_fold_2_seed_0', '/home/jupyter/karman-project/kml_results/kml_fold_2_seed_1']
Loading Data
Namespace(batch_size=256, cyclical_features=True, data_directory='/home/jupyter/karman-project/data_directory', dropout=0.0, epochs=10, epochs_per_validation=1, features_to_exclude_fism2_daily_stan_bands='all__dates_datetime__', features_to_exclude_fism2_flare_stan_bands='all__dates_datetime__', features_to_exclude_omni='all__dates_datetime__,omniweb__id_for_imf_spacecraft__,omniweb__id_for_sw_plasma_spacecraft__,omniweb__#_of_points_in_imf_averages__,omniweb__#_of_points_in_plasma_averages__,omniweb__percent_of_interpolation__,omniweb__timeshift__[s],omniweb__rms_timeshift__[s],omniweb__rms_min_variance__[s**2],omniweb__time_btwn_observations__[s],omniweb__rms_sd_b_scalar__[nT],omniweb__rms_sd_b_field_vector__[nT],omniweb__flow_pressure__[nPa],omniweb__electric_field__[mV/m],omniweb__plasma_beta__,omniweb__alfven_mach_number__,omniwe

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '



Finished Creating dataset.
Loading Model
Creating training, validation and test sets.


17 years to iterate through.: 100%|██████████| 17/17 [01:46<00:00,  6.24s/it]


Train size: 27719788
Validation size: 3011830
Test size: 2575188


...

17 years to iterate through.: 100%|██████████| 17/17 [01:45<00:00,  6.23s/it]


Train size: 27719788
Validation size: 3011830
Test size: 2575188


100%|██████████| 5030/5030 [00:12<00:00, 394.69it/s]


Ignoring outliers...
Removed 14 entries with outlier densities below 1e-17
Evaluating Storm Condition Results.
Saving results to /home/jupyter/karman-project/benchmark_results/model_results.csv


(                                    Metric Value
 Model Metric Type Condition                     
 JB08  MAPE        1. (0-15) Quiet   4.346949e+01
                   2. (15-30) Mild   2.460980e+01
                   200km-250km       1.475357e+01
                   250km-300km       2.230136e+01
                   3. (30-50) Minor  2.381358e+01
 ...                                          ...
 model RMSE        4. (50+) Major    1.858496e-11
                   400km-450km       1.064800e-13
                   450km-500km       7.869352e-14
                   500km-550km       4.224416e-14
                   No condition      3.163810e-12
 
 [72 rows x 1 columns],
                                     Metric Value
 Model Metric Type Condition                     
 JB08  MAPE        1. (0-15) Quiet            0.0
                   2. (15-30) Mild            0.0
                   200km-250km                0.0
                   250km-300km                0.0
                   3. (3